In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

In [63]:
import tensorflow as tf
import tf2onnx

## Reading the data

In [3]:
data = pd.read_csv(r"..\..\Data\small_ohe.csv")
data.head()

,age,default,housing,loan,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,...,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
0,0.171429,1,-1,1,0.029412,1.0,0.0,0,0.333333,0.269680,...,0,1,0,0,0,1,0,0,0,0
1,0.300000,1,1,1,0.088235,1.0,0.0,0,0.937500,0.698753,...,0,1,0,0,0,1,0,0,0,0
2,0.100000,1,-1,1,0.000000,1.0,0.0,0,1.000000,0.882307,...,0,0,0,0,0,0,0,0,0,1
3,0.285714,1,0,0,0.058824,1.0,0.0,0,1.000000,0.882307,...,0,0,0,0,0,1,0,0,0,0
4,0.414286,1,-1,1,0.000000,1.0,0.0,0,0.687500,0.389322,...,0,0,1,0,0,0,1,0,0,0


In [4]:
data.columns

Index(['age', 'default', 'housing', 'loan', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'y', 'pdays2', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_w

In [5]:
# Saperating features and result vectors
X = data.drop('y', axis=1).values
y = data['y'].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [59]:
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(55,),use_bias=True),
    tf.keras.layers.Dense(64, activation='relu',use_bias=True),
    tf.keras.layers.Dense(32, activation='relu',use_bias=True),
    tf.keras.layers.Dense(1, activation='sigmoid',use_bias=True)
])
model.compile(loss='BinaryCrossentropy', optimizer='adam', metrics=['accuracy'])

In [60]:
model.fit(X_train, y_train, batch_size=128, epochs=60, validation_data=(X_test, y_test))

Epoch 1/60
40/40 [==============================] - 1s 3ms/step - loss: 0.6363 - accuracy: 0.6682 - val_loss: 0.5572 - val_accuracy: 0.7407
Epoch 2/60
40/40 [==============================] - 0s 1ms/step - loss: 0.5484 - accuracy: 0.7308 - val_loss: 0.5307 - val_accuracy: 0.7531
Epoch 3/60
40/40 [==============================] - 0s 1ms/step - loss: 0.5233 - accuracy: 0.7410 - val_loss: 0.5136 - val_accuracy: 0.7572
Epoch 4/60
40/40 [==============================] - 0s 1ms/step - loss: 0.5033 - accuracy: 0.7557 - val_loss: 0.4969 - val_accuracy: 0.7645
Epoch 5/60
40/40 [==============================] - 0s 1ms/step - loss: 0.4787 - accuracy: 0.7764 - val_loss: 0.4869 - val_accuracy: 0.7710
Epoch 6/60
40/40 [==============================] - 0s 1ms/step - loss: 0.4563 - accuracy: 0.7891 - val_loss: 0.4601 - val_accuracy: 0.7865
Epoch 7/60
40/40 [==============================] - 0s 1ms/step - loss: 0.4275 - accuracy: 0.8074 - val_loss: 0.4438 - val_accuracy: 0.7980
Epoch 8/60
40/40 [==

In [61]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_66 (Dense)            (None, 64)                3584      
                                                                 
 dense_67 (Dense)            (None, 64)                4160      
                                                                 
 dense_68 (Dense)            (None, 32)                2080      
                                                                 
 dense_69 (Dense)            (None, 1)                 33        
                                                                 
Total params: 9,857
Trainable params: 9,857
Non-trainable params: 0
_________________________________________________________________


## Convert NN to ONNX file

In [64]:
# convert the model to ONNX format
onnx_model, _ = tf2onnx.convert.from_keras(model)

# save the ONNX model to a file
with open("my_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## Setup onnx

In [68]:
import onnx
from onnx import numpy_helper
from onnx_tf.backend import prepare

onnx.load('./my_model.onnx')

# def setup(onnx_file: str,):
#     # Load the ONNX model
#     model = onnx.load(model_path)
    
#     # Extract the model parameters
#     params = {}
#     for init in model.graph.initializer:
#         name = init.name
#         value = numpy_helper.to_array(init)
#         params[name] = value
    
#     # Print the summary of the parameters
#     for name, value in params.items():
#         print("Parameter: {0}, shape: {1}, min: {2:.2f}, max: {3:.2f}, mean: {4:.2f}, std: {5:.2f}"
#               .format(name, value.shape, value.min(), value.max(), value.mean(), value.std()))

ir_version: 7
producer_name: "tf2onnx"
producer_version: "1.13.0 2c1db5"
graph {
  node {
    input: "dense_66_input"
    input: "sequential_20/dense_66/MatMul/ReadVariableOp:0"
    output: "sequential_20/dense_66/MatMul:0"
    name: "sequential_20/dense_66/MatMul"
    op_type: "MatMul"
  }
  node {
    input: "sequential_20/dense_66/MatMul:0"
    input: "sequential_20/dense_66/BiasAdd/ReadVariableOp:0"
    output: "sequential_20/dense_66/BiasAdd:0"
    name: "sequential_20/dense_66/BiasAdd"
    op_type: "Add"
  }
  node {
    input: "sequential_20/dense_66/BiasAdd:0"
    output: "sequential_20/dense_66/Relu:0"
    name: "sequential_20/dense_66/Relu"
    op_type: "Relu"
  }
  node {
    input: "sequential_20/dense_66/Relu:0"
    input: "sequential_20/dense_67/MatMul/ReadVariableOp:0"
    output: "sequential_20/dense_67/MatMul:0"
    name: "sequential_20/dense_67/MatMul"
    op_type: "MatMul"
  }
  node {
    input: "sequential_20/dense_67/MatMul:0"
    input: "sequential_20/dense_67/Bi

In [69]:
tf_model = prepare(model)

AttributeError: 'Sequential' object has no attribute 'SerializeToString'

In [ ]:
## test confidence function